<a href="https://colab.research.google.com/github/linhlinhle997/low-resource-nmt-bart/blob/develop/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import torch
from transformers import (
    T5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoModelWithLMHead,
    pipeline
)

## Text Classification

The model `mrm8488/t5-small-finetuned-imdb-sentiment` is a T5-small model fine-tuned for sentiment analysis on the IMDB dataset.

In [30]:
model_name = 'mrm8488/t5-small-finetuned-imdb-sentiment'

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [31]:
input = "i like this movie"
label = "positive"

In [32]:
# Tokenize input
encoded_input = tokenizer(input, return_tensors='pt')
encoded_input

{'input_ids': tensor([[   3,   23,  114,   48, 1974,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [33]:
# Tokenize label
encoded_label = tokenizer(label, return_tensors='pt')
encoded_label

{'input_ids': tensor([[1465,    1]]), 'attention_mask': tensor([[1, 1]])}

In [34]:
# Compute loss and logits for training
logit = model(
    input_ids=encoded_input.input_ids,
    labels=encoded_label.input_ids
)

print(f"Loss: {logit.loss.item()}")
print(f"Logits shape: {logit.logits.shape}")

Loss: 0.0017512135673314333
Logits shape: torch.Size([1, 2, 32128])


In [35]:
# Generate output (predict sentiment label)
pred = model.generate(
    encoded_input.input_ids,
    attention_mask=encoded_input.attention_mask,
    max_length=2
)
pred

tensor([[   0, 1465]])

In [36]:
decoded_output = tokenizer.decode(pred[0], skip_special_tokens=True)
print(decoded_output)

positive


## Translation

The model `t5-small` is a general-purpose text-to-text transformer trained by Google (summarization, translation, QA, and text generation).

In [37]:
model_name = 't5-small'

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [38]:
# Define input, output text
input = 'I go to school'
output = 'Ich besuche die Schule'

In [39]:
# Add prompt for translation task
input = "translate English to German: " + input
input

'translate English to German: I go to school'

In [40]:
# Tokenize input
encoded_input = tokenizer(input, return_tensors='pt')
encoded_input

{'input_ids': tensor([[13959,  1566,    12,  2968,    10,    27,   281,    12,   496,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [41]:
# Tokenize output
encoded_label = tokenizer(label, return_tensors='pt')
encoded_input

{'input_ids': tensor([[13959,  1566,    12,  2968,    10,    27,   281,    12,   496,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [42]:
# Compute loss and logits
logit = model(
    input_ids=encoded_input.input_ids,
    labels=encoded_label.input_ids
)

print(f"Loss: {logit.loss.item()}")
print(f"Logits shape: {logit.logits.shape}")

Loss: 11.970788955688477
Logits shape: torch.Size([1, 2, 32128])


In [43]:
# Generate translation
pred = model.generate(
    encoded_input.input_ids,
    attention_mask=encoded_input.attention_mask,
    max_length=20
)
pred

tensor([[    0,  1674,     3, 27024,    15,    67, 12853,     1]])

In [44]:
# Decode generated text
translated_text = tokenizer.batch_decode(pred, skip_special_tokens=True)

print(f"Translated Output: {translated_text[0]}")

Translated Output: Ich besuche die Schule


## Sumarization

In [45]:
document = """
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.
"""

In [46]:
# Prepare input for summarization
input = "summarize: " + document
input

'summarize: \nLONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

In [47]:
tokenizer.model_max_length

512

In [51]:
# Tokenizer input
encoded_input = tokenizer(input, return_tensors='pt', max_length=tokenizer.model_max_length, truncation=True)

In [52]:
# Generate summary
pred = model.generate(
    encoded_input.input_ids,
    attention_mask=encoded_input.attention_mask,
    max_length=200
)

In [59]:
summary = tokenizer.batch_decode(pred, skip_special_tokens=True)

summary[0]

'he says he has no plans to fritter cash away on fast cars, drink and celebrity parties. the actor says he\'ll be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II" he has filmed a television movie about author Rudyard Kipling and his son.'

## Question Answering

In [61]:
model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1881: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [62]:
# Define question and context
question = "What is 42?"
context = "42 is the answer to life, the universe and everything"

In [63]:
# Format input for the model
input_text = f"question: {question} context: {context}"

# Tokenizer input
encoded_input = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)

In [64]:
# Generate answer
output = model.generate(input_ids=encoded_input.input_ids, attention_mask=encoded_input.attention_mask)

output

tensor([[    0,    37,  1525,    12,   280,     6,     8,  8084,     6,    11,
           762,    16,    34,    31,     7, 26771,     5,     1]])

In [66]:
# Decode answer
answer = tokenizer.decode(output[0], skip_special_tokens=True)

print(answer)

The answer to life, the universe, and everything in it's entirety.
